In [1]:
print("hello")

hello


In [3]:
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.llms import ctransformers
from langchain_pinecone import PineconeVectorStore

c:\Users\krish\projects\med-bot\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
from dotenv import load_dotenv
load_dotenv()
import os

In [10]:
pinecone_api=os.getenv("PINECONE_API_KEY")


In [8]:
def importdata(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    document=loader.load()
    return document

In [9]:
docs=importdata("data/")

In [10]:
def datachunking(data_extracted):
    chunker=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
    chunks=chunker.split_documents(data_extracted)
    return chunks

In [11]:
texts=datachunking(docs)

In [12]:
len(texts)

30789

In [1]:
def embeddingmodel():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [4]:
embedder=embeddingmodel()

In [11]:

os.environ['PINECONE_API_KEY'] = pinecone_api
#PineconeVectorStore.from_texts([p.page_content for p in texts],embedding=embedder,index_name="med-bot")

In [12]:
from langchain_pinecone import PineconeVectorStore
import os

docsearch=PineconeVectorStore.from_existing_index(index_name="med-bot",embedding=embedder)

In [13]:
res=docsearch.similarity_search("what is acne?",k=2)

In [14]:
print(res[0].page_content,res[1].page_content)

infect sebaceous follicles, causing acne.Pustule— A pus-filled pimple.
Sebaceous follicles— Structures in the skin that
contain oil-producing glands and hair follicles
and which give rise acne.
Sebum— An oily skin moisturizer produced by
sebaceous glands.
Tretinoin— A naturally occurring retinoid, derived
from vitamin A, that treats acne by increasing the
turnover (death and replacement) of skin cells.
GALE ENCYCLOPEDIA OF MEDICINE, 5THEDITION 27Acne disease
Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin condition characterized by
pimples on the face, chest, and back. It occurs when
the pores of the skin become clogged with oil, deadskin cells, and bacteria.
Demographics
Acne vulgaris , or common acne, is the most prev-
alent of all skin diseases. It affects nearly 17 millionpeople in the United States. Nearly 85% of youngpeople develop acne at some time between the agesof 12 and 25 years. It usually begins at puberty 

In [15]:
prompttemplate="""
use the following pieces of context to answer the user's question
if you don't know the answer just say "not related to me" ,don't try to make up the answer
context={context}
question={question}
summerize the result with right words and return below
answer:""" 
pprompt=PromptTemplate(template=prompttemplate,input_variables=["context", "question"])

In [17]:
llm=ctransformers.CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={"max_new_tokens":512,
                          "temperature":0.5})

In [18]:

medbot=RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=docsearch.as_retriever(kwargs={"k":2}),
                              
                            chain_type_kwargs={"prompt": pprompt})

In [19]:
response=medbot({"query":"what is acne?"})
print(response["result"])


c:\Users\krish\projects\med-bot\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Acneverification.
Acneverified
related to the user'
not related to the users question in seconds later.
Acneedsinception:
not related to the user'
Acneither related to the user'
Acneitherelated, I am not related to the following a question 1: Acneverified.
related to the user's to the user'the users query, if you are based on your question 1: Acneverified by typing "not related to the user'The word count=Acne of context or response
Not related to the user's
Acneedscientistically related to the user'
Acneedsinception
related to the users question
Acneous
Sebace, please.
Acne
/
Acneverified
not related to the following words that is related to the user'If you
Not related to the user'
Related.
Acneverbat
Acne
acneverification
related to the users question  Acne
Acneither related to the user'
Acneverified
Acneedsinception.
not related to the user'
Acneverdict.
Acneverified.
Not related to the following
Acneverified by the user'the user's
related to the user question
Acneverbat
Acneverific

In [20]:
medbot.invoke("acne")

In [24]:
from langchain.chains import RetrievalQAWithSourcesChain
medbot2=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=docsearch.as_retriever())

In [ ]:
medbot2({"question":"what is aging"},return_only_outputs=True)